# Setup

In [467]:
# Installs
# %pip install geopandas
# %pip install folium  
# %pip install plotly==5.11.0
# % pip install dash
# % pip install plotly.express

In [468]:
# Imports
import pandas as pd
import streamlit as st
import numpy as np
import matplotlib.pyplot as plt

import chart_studio.plotly as py
import plotly.express as px
import plotly.offline as po
import plotly.graph_objs as pg


In [469]:
# Pandas options
pd.set_option('display.max_rows', 30) # Display 70 rows
pd.set_option('display.float_format', lambda x: '%.5f' % x) # Suppress scientific notation in Pandas

# Data

## Import data

In [470]:
# Data source: World Bank

# Population
# https://data.worldbank.org/indicator/SP.POP.TOTL
df_population_raw = pd.read_csv('Data Population/API_SP.POP.TOTL_DS2_en_csv_v2_4770387.csv', skiprows=4)

# Individuals using the Internet (% of population) = IT.NET.USER.ZS
# https://data.worldbank.org/indicator/IT.NET.USER.ZS
df_users_raw = pd.read_csv('Data internet/individuals_using_the_Internet_percentage_of_population.csv', skiprows=4)
df_metadata_country_raw = pd.read_csv('Data internet/Metadata_Country_individuals_using_the_Internet_percentage_of_population.csv')
df_metadata_indicator_raw= pd.read_csv('Data internet/Metadata_Indicator_individuals_using_the_Internet_percentage_of_population.csv')

In [471]:
df_users_1 = df_users_raw.copy()
df_metadata_country = df_metadata_country_raw.copy()
df_metadata_indicator = df_metadata_indicator_raw.copy()
df_population = df_population_raw.copy()

## Clean data

### Population

In [472]:
# Drop unnecessary columns
drop_columns_population = ['Indicator Name', 'Indicator Code', 'Unnamed: 66']
df_population.drop(columns = drop_columns_population, inplace = True)

In [473]:
# Wide to long
# Get years columns
years = df_population.columns[2:]

# Use melt to unpivot the DataFrame
df_population = df_population.melt(id_vars=['Country Name', 'Country Code'], value_vars=years, var_name='Year', value_name='SP.POP.TOTL')

In [474]:
# Sort by country and year
df_population.sort_values(by=['Country Name', 'Year'], inplace=True)

In [475]:
# Rename columns
df_population.rename(columns={"SP.POP.TOTL": "Population"}, inplace = True)

In [476]:
# To-do

# Correct dadatypes
# Ugly alternative
df_population['Year'] = df_population['Year'].astype(int)

In [477]:
df_population.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16492 entries, 2 to 16491
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Country Name  16492 non-null  object 
 1   Country Code  16492 non-null  object 
 2   Year          16492 non-null  int64  
 3   Population    16400 non-null  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 644.2+ KB


### Users

In [478]:
# Drop unnecessary columns
drop_columns_users = ['Indicator Name', 'Indicator Code', 'Unnamed: 66']
df_users_1.drop(columns = drop_columns_users, inplace = True)

In [479]:
# Wide to long
# Get years columns
years = df_users_1.columns[2:]

# Use melt to unpivot the DataFrame
df_users_1 = df_users_1.melt(id_vars=['Country Name', 'Country Code'], value_vars=years, var_name='Year', value_name='IT.NET.USER.ZS')

In [480]:
# Sort by country and year
df_users_1.sort_values(by=['Country Name', 'Year'], inplace=True)

In [481]:
# Rename columns
df_users_1.rename(columns={"IT.NET.USER.ZS": "Users percentage"}, inplace = True)

In [482]:
# To-do

# Correct dadatypes
# I cn't make this work correctly
# df_users_1['Year'] = pd.to_datetime(df_users_1['Year'], format='%Y')

# Ugly alternative
df_users_1['Year'] = df_users_1['Year'].astype(int)

In [483]:
df_users_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16492 entries, 2 to 16491
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Country Name      16492 non-null  object 
 1   Country Code      16492 non-null  object 
 2   Year              16492 non-null  int64  
 3   Users percentage  7749 non-null   float64
dtypes: float64(1), int64(1), object(2)
memory usage: 644.2+ KB


In [484]:
# Decide on timespan to analize
which_years_to_keep = df_users_1.groupby('Year')['Users percentage'].count().reset_index()

# VIZ Year vs IT.NET.USER.ZS
fig = px.line(which_years_to_keep, x="Year", y="Users percentage", title="Individuals using the Internet (% of population)")
fig.show()

In [485]:
# Keep data from 1990 to 2020
df_users_1 = df_users_1[df_users_1['Year'].between(1990, 2020, inclusive='both')]
df_users_1

,Country Name,Country Code,Year,Users percentage
7982,Afghanistan,AFG,1990,0.00000
8248,Afghanistan,AFG,1991,0.00000
8514,Afghanistan,AFG,1992,0.00000
8780,Afghanistan,AFG,1993,0.00000
9046,Afghanistan,AFG,1994,0.00000
...,...,...,...,...
15161,Zimbabwe,ZWE,2016,23.11999
15427,Zimbabwe,ZWE,2017,24.40000
15693,Zimbabwe,ZWE,2018,25.00000
15959,Zimbabwe,ZWE,2019,25.10000


### metadata_country

In [486]:
df_metadata_country.head(3)

,Country Code,Region,IncomeGroup,SpecialNotes,TableName,Unnamed: 5
0,ABW,Latin America & Caribbean,High income,NaN,Aruba,NaN
1,AFE,NaN,NaN,"26 countries, stretching from the Red Sea in t...",Africa Eastern and Southern,NaN
2,AFG,South Asia,Low income,The reporting period for national accounts dat...,Afghanistan,NaN


In [487]:
# Drop unnecessary columns
drop_columns_metadata_country = ['TableName', 'Unnamed: 5']
df_metadata_country.drop(columns = drop_columns_metadata_country, inplace=True)


In [488]:
df_metadata_country.head(3)

,Country Code,Region,IncomeGroup,SpecialNotes
0,ABW,Latin America & Caribbean,High income,NaN
1,AFE,NaN,NaN,"26 countries, stretching from the Red Sea in t..."
2,AFG,South Asia,Low income,The reporting period for national accounts dat...


### metadata_indicator

In [489]:
df_metadata_indicator.head()

,INDICATOR_CODE,INDICATOR_NAME,SOURCE_NOTE,SOURCE_ORGANIZATION,Unnamed: 4
0,IT.NET.USER.ZS,Individuals using the Internet (% of population),Internet users are individuals who have used t...,International Telecommunication Union (ITU) Wo...,NaN


### Merge users_1 and metadata

In [490]:
# Merge users and metadata
users_metadata = df_users_1.merge(df_metadata_country, how='left', on='Country Code')
users_metadata.head(3)


,Country Name,Country Code,Year,Users percentage,Region,IncomeGroup,SpecialNotes
0,Afghanistan,AFG,1990,0.00000,South Asia,Low income,The reporting period for national accounts dat...
1,Afghanistan,AFG,1991,0.00000,South Asia,Low income,The reporting period for national accounts dat...
2,Afghanistan,AFG,1992,0.00000,South Asia,Low income,The reporting period for national accounts dat...


In [491]:
users_metadata[users_metadata['Country Code'] == 'WLD']

,Country Name,Country Code,Year,Users percentage,Region,IncomeGroup,SpecialNotes
8122,World,WLD,1990,0.04882,NaN,NaN,World aggregate.
8123,World,WLD,1991,0.07881,NaN,NaN,World aggregate.
8124,World,WLD,1992,0.12519,NaN,NaN,World aggregate.
8125,World,WLD,1993,0.17901,NaN,NaN,World aggregate.
8126,World,WLD,1994,0.36051,NaN,NaN,World aggregate.
...,...,...,...,...,...,...,...
8148,World,WLD,2016,43.28537,NaN,NaN,World aggregate.
8149,World,WLD,2017,45.76617,NaN,NaN,World aggregate.
8150,World,WLD,2018,49.23364,NaN,NaN,World aggregate.
8151,World,WLD,2019,53.57066,NaN,NaN,World aggregate.


In [492]:
# Replace values in column 'Region' with 'World where 'Region' is 'North'
users_metadata.loc[users_metadata['Country Name'] == 'World', 'Region'] = 'World'

In [493]:
# Save to file and re-import
users_metadata.to_csv('users_metadata.csv', index=False)

# Usable dataframe
df_users_metadata = pd.read_csv('users_metadata.csv')

In [494]:
users_metadata.head(3)

,Country Name,Country Code,Year,Users percentage,Region,IncomeGroup,SpecialNotes
0,Afghanistan,AFG,1990,0.00000,South Asia,Low income,The reporting period for national accounts dat...
1,Afghanistan,AFG,1991,0.00000,South Asia,Low income,The reporting period for national accounts dat...
2,Afghanistan,AFG,1992,0.00000,South Asia,Low income,The reporting period for national accounts dat...


In [495]:
users_metadata.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8246 entries, 0 to 8245
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Country Name      8246 non-null   object 
 1   Country Code      8246 non-null   object 
 2   Year              8246 non-null   int64  
 3   Users percentage  7548 non-null   float64
 4   Region            6758 non-null   object 
 5   IncomeGroup       6696 non-null   object 
 6   SpecialNotes      3906 non-null   object 
dtypes: float64(1), int64(1), object(5)
memory usage: 515.4+ KB


### Merge  users_metadata & df_population

In [496]:
# Merge users and metadata
users_population = users_metadata.merge(df_population, how='left', on=['Country Name', 'Country Code', 'Year'])

In [497]:
# Add column 'Users Total': total number of people using the internet
# The total number of people using the internet is calculated by multiplying the 
# % of population using the Internet['Users percentage']  
# with the population estimate ['Population']

users_population['Users Total'] = (users_population['Users percentage'] * users_population['Population']) / 100

In [498]:
# Save to file and re-import
users_population.to_csv('users.csv', index=False)

# Usable dataframe
df_users = pd.read_csv('users.csv')

In [499]:
df_users[df_users['Country Code'] == 'USA'].tail()

,Country Name,Country Code,Year,Users percentage,Region,IncomeGroup,SpecialNotes,Population,Users Total
7869,United States,USA,2016,85.54442,North America,High income,NaN,323071755.00000,276369863.16620
7870,United States,USA,2017,87.27489,North America,High income,NaN,325122128.00000,283749976.87915
7871,United States,USA,2018,88.49890,North America,High income,NaN,326838199.00000,289248221.25558
7872,United States,USA,2019,89.43028,North America,High income,NaN,328329953.00000,293626412.24860
7873,United States,USA,2020,90.90000,North America,High income,NaN,331501080.00000,301334481.72000


In [500]:
df_users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8246 entries, 0 to 8245
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Country Name      8246 non-null   object 
 1   Country Code      8246 non-null   object 
 2   Year              8246 non-null   int64  
 3   Users percentage  7548 non-null   float64
 4   Region            6758 non-null   object 
 5   IncomeGroup       6696 non-null   object 
 6   SpecialNotes      3906 non-null   object 
 7   Population        8215 non-null   float64
 8   Users Total       7548 non-null   float64
dtypes: float64(3), int64(1), object(5)
memory usage: 579.9+ KB


# Analisis and VIZ

In [501]:
# Create df_regions and df_income_group
df_regions = df_users.groupby(['Region', 'Year']).agg({'Users percentage':'mean', 'Users Total': 'sum'}).reset_index()
df_income_group = df_users.groupby(['IncomeGroup', 'Year']).agg({'Users percentage':'mean', 'Users Total': 'sum'}).reset_index()

## Share of the population using the internet by country, historical progression

In [502]:
fig = px.choropleth(df_users,
                    locations="Country Code",
                    color="Users percentage",
                    hover_name="Country Name", # column to add to hover information
                    color_continuous_scale=px.colors.sequential.Plasma,
                    animation_frame="Year",
                    animation_group="Country Name",
                    )
fig.show()

## Share of the population using the internet by country

In [503]:
# VIZ Variables
title = df_metadata_indicator['INDICATOR_NAME']
note = df_metadata_indicator['SOURCE_NOTE']
source = df_metadata_indicator['SOURCE_ORGANIZATION']

y_max = df_users['Users percentage'].max() + 10

In [504]:
# To-do : add a line representing the world progression over time in a different colour
# To-do : add year selector on dash
# To-do : Style VIZ

# VIZ Share of the population using the internet
# https://plotly.com/python/line-charts/

fig = px.line(df_users,
              x="Year",
              y="Users percentage",
              color='Country Name',
#              text=df_users['Year']
            )
fig.show()

## Share of the population using the internet by Income Group

In [505]:
# VIZ Share of the population using the internet
# https://plotly.com/python/line-charts/

fig = px.line(df_income_group,
              x="Year",
              y="Users percentage",
              color='IncomeGroup',
#              text=df_users['Year']
            )
fig.show()



## Share of the population using the internet by Region

In [506]:
fig = px.line(df_regions,
              x="Year",
              y="Users percentage",
              color='Region',
#              text=df_users['Year']
            )
fig.show()


## Total number of people using the internet by region

In [507]:
fig = px.line(df_regions,
              x="Year",
              y="Users Total",
              color='Region',
#              text=df_users['Year']
            )
fig.show()

## Top 10 countries with the highest internet use (by population share) in 2020?

In [508]:
# Top 10 countries with the highest internet use by population share in 2020

year_max = df_users['Year'].max()
df_top_10_2020 = df_users.query(f'Year == {year_max}')

df_top_10_2020 = df_top_10_2020.groupby(['Country Name', 'Country Code','Year' ])['Users percentage'].sum().to_frame().reset_index()
df_top_10_2020 = df_top_10_2020.sort_values(by=['Year', 'Users percentage'], ascending=False)[:10]
df_top_10_2020

,Country Name,Country Code,Year,Users percentage
251,United Arab Emirates,ARE,2020,100.00000
17,Bahrain,BHR,2020,99.67105
200,Qatar,QAT,2020,99.65285
128,Kuwait,KWT,2020,99.10588
110,Iceland,ISL,2020,99.00000
146,Luxembourg,LUX,2020,98.82242
207,Saudi Arabia,SAU,2020,97.86233
182,Norway,NOR,2020,97.00000
58,Denmark,DNK,2020,96.54915
126,"Korea, Rep.",KOR,2020,96.50506


In [509]:
# Documentation of this VIZ "Using Built-in Country and State Geometries":
# https://plotly.com/python/choropleth-maps/

fig = px.choropleth(df_top_10_2020,
                    locations="Country Code",
                    color="Users percentage",
                    hover_name="Country Name", # column to add to hover information
                    color_continuous_scale=px.colors.sequential.Plasma)
fig.show()

## Top 10 over time

In [510]:
# Top 10 countries with the highest internet use by population share over time

def top_10(my_df, col_year):
    years = my_df[col_year].unique()

    df_top_10 = pd.DataFrame(columns=['Country Name', 'Country Code',  'Year',  'Users percentage'])

    for i in years:
        ds = my_df.query(f'Year == {i}')
        ds = ds.groupby(['Country Name', 'Country Code','Year' ])['Users percentage'].sum().to_frame().reset_index()
        ds = ds.sort_values(by=['Users percentage'], ascending=False)[:10]
        df_top_10 = pd.concat([df_top_10, ds])
    return df_top_10



In [511]:
df_top_10 = top_10(df_users, 'Year')
df_top_10

,Country Name,Country Code,Year,Users percentage
253,United States,USA,1990,0.78473
179,North America,NAC,1990,0.74207
182,Norway,NOR,1990,0.70730
235,Switzerland,CHE,1990,0.59571
13,Australia,AUS,1990,0.58509
...,...,...,...,...
146,Luxembourg,LUX,2020,98.82242
207,Saudi Arabia,SAU,2020,97.86233
182,Norway,NOR,2020,97.00000
58,Denmark,DNK,2020,96.54915


In [512]:
fig = px.choropleth(df_top_10,
                    locations="Country Code",
                    color="Users percentage",
                    hover_name="Country Name", # column to add to hover information
                    color_continuous_scale=px.colors.sequential.Plasma,
                    animation_frame="Year",
                    animation_group="Country Name",
                    )
fig.show()